In [1]:
!pip install python-twitter
!pip install pandas

You should consider upgrading via the 'C:\Users\colacan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\colacan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
twitter_consumer_key = "o5RkzZCWRv82BO3Q7Ou6g3gP6"
twitter_consumer_secret = "ZkhVNVnl2AbiXcQNfFg3nyUujC8Im3dsaLWkTgtxY8bPMFWnFm"  
twitter_access_token = "1511161028247138313-qJlKodPZHwX6CNHKhfkrVBRO57pj92"
twitter_access_secret = "CTkfZ2vMesebR87k7i46T7qcSUqdOIv5oPmmjNS8XFxnX"
import twitter
twitter_api = twitter.Api(consumer_key=twitter_consumer_key,
                          consumer_secret=twitter_consumer_secret, 
                          access_token_key=twitter_access_token, 
                          access_token_secret=twitter_access_secret)

In [26]:
query = "리그오브레전드"
statuses = twitter_api.GetSearch(term=query, count=100)

In [27]:
import pandas as pd
df = pd.DataFrame()

In [28]:
for status in statuses:
    new_df = pd.DataFrame([status.text])
    df = pd.concat([df,new_df])

In [29]:
df

,0
0,RT @noname_46431: RT❤\n리그오브레전드(LOL) 장르의 모데카이저 ...
0,RT @Hawool_99: 리그오브레전드 롤 라칸(아무스킨이나) 코스프레 의상 구합...
0,리그오브레전드 롤 라칸(아무스킨이나) 코스프레 의상 구합니다~~!!! \n\n+같은...
0,RT @pepsizero_joa: 2022 PlayX4 \n\n에이크라운 부스\n\...
0,RT @COSHS_GORAE: #플엑_트친소\n#플레이엑스포_트친소\n#코스어_트친...
...,...
0,RT @COSHS_GORAE: #플엑_트친소\n#플레이엑스포_트친소\n#코스어_트친...
0,RT @COSHS_GORAE: #플엑_트친소\n#플레이엑스포_트친소\n#코스어_트친...
0,롤RP대리결제 롤RP대리충전이 아닌 안전한 롤RP선물식 판매합니다 6725RP 13...
0,RT @COSHS_GORAE: #플엑_트친소\n#플레이엑스포_트친소\n#코스어_트친...
